# Capstone 3: Deep Neural Network Modeling & Prediction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import loadtxt

In [2]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [3]:
from keras.models import Sequential
from keras.layers import Dense

In [4]:
import tensorflow as ts

In [5]:
df_train = pd.read_csv('train.csv')

In [6]:
df_test = pd.read_csv('test.csv')

In [7]:
# off the bat we can drop the patients, and parents names, as well as the institue and the name of the institute as 
# this does not affect genetic outcome
df_train.drop(['Patient Id', 'Patient First Name', 'Family Name', "Father's name",
           'Institute Name', 'Institute Name', 'Location of Institute', 'Status', 'Place of birth',
           ], axis = 1, inplace = True)

train = df_train.rename(columns = {'Patient Age':'Patient_Age', "Genes in mother's side":"Genes_in_mothers_side",
            'Inherited from father':'Inherited_from_father',
            'Maternal gene':'Maternal_gene', 'Paternal gene':'Paternal_gene',
            'Blood cell count (mcL) ':'Blood_cell_count', "Father's age":'Fathers_age', "Mother's age":'Mothers_age',
            'Respiratory Rate (breaths/min)':'Respiratory_Rate', 'Heart Rate (rates/min)':'Heart_Rate',
            'Test 1':'Test_1', 'Test 2':'Test_2', 'Test 3':'Test_3',
            'Test 4':'Test_4', 'Test 5':'Test_5', 'Parental consent': 'Parental_consent', 'Follow-up':'Follow_up', 
             'Birth asphyxia':'Birth_asphyxia',
             'Autopsy shows birth defect (if applicable)': 'Autopsy_shows_birth_defect', 
             'Folic acid details (peri-conceptional)':'Folic_acid_details', 
            'H/O serious maternal illness':'HO_serious_maternal_illness',
            'H/O radiation exposure (x-ray)':'HO_radiation_exposure', 'H/O substance abuse':'HO_substance_abuse',
            'Assisted conception IVF/ART':'Assisted_conception_IVFART', 
            'History of anomalies in previous pregnancies':'History_of_anomalies_in_previous_pregnancies',
            'No. of previous abortion':'No_of_previous_abortion', 'Birth defects':'Birth_defects', 
            'White Blood cell count (thousand per microliter)':'White_Blood_cell_count', 
            'Blood test result':'Blood_test_result',
            'Symptom 1':'Symptom_1', 'Symptom 2':'Symptom_2', 'Symptom 3':'Symptom_3', 'Symptom 4':'Symptom_4', 
            'Symptom 5':'Symptom_5',
            'Genetic Disorder':'Genetic_Disorder', 'Disorder Subclass':'Disorder_Subclass', 
            'Blood cell count (mcL)': 'Blood_cell_count'})

In [8]:
df_test.drop(['Patient Id', 'Patient First Name', 'Family Name', "Father's name",
           'Institute Name', 'Institute Name', 'Location of Institute', 'Status', 'Place of birth',
           ], axis = 1, inplace = True)

test = df_test.rename(columns = {'Patient Age':'Patient_Age', "Genes in mother's side":"Genes_in_mothers_side",
            'Inherited from father':'Inherited_from_father',
            'Maternal gene':'Maternal_gene', 'Paternal gene':'Paternal_gene',
            'Blood cell count (mcL) ':'Blood_cell_count', "Father's age":'Fathers_age', "Mother's age":'Mothers_age',
            'Respiratory Rate (breaths/min)':'Respiratory_Rate', 'Heart Rate (rates/min)':'Heart_Rate',
            'Test 1':'Test_1', 'Test 2':'Test_2', 'Test 3':'Test_3',
            'Test 4':'Test_4', 'Test 5':'Test_5', 'Parental consent': 'Parental_consent', 'Follow-up':'Follow_up', 
             'Birth asphyxia':'Birth_asphyxia',
             'Autopsy shows birth defect (if applicable)': 'Autopsy_shows_birth_defect', 
             'Folic acid details (peri-conceptional)':'Folic_acid_details', 
            'H/O serious maternal illness':'HO_serious_maternal_illness',
            'H/O radiation exposure (x-ray)':'HO_radiation_exposure', 'H/O substance abuse':'HO_substance_abuse',
            'Assisted conception IVF/ART':'Assisted_conception_IVFART', 
            'History of anomalies in previous pregnancies':'History_of_anomalies_in_previous_pregnancies',
            'No. of previous abortion':'No_of_previous_abortion', 'Birth defects':'Birth_defects', 
            'White Blood cell count (thousand per microliter)':'White_Blood_cell_count', 
            'Blood test result':'Blood_test_result',
            'Symptom 1':'Symptom_1', 'Symptom 2':'Symptom_2', 'Symptom 3':'Symptom_3', 'Symptom 4':'Symptom_4', 
            'Symptom 5':'Symptom_5',
            'Genetic Disorder':'Genetic_Disorder', 'Disorder Subclass':'Disorder_Subclass', 
            'Blood cell count (mcL)': 'Blood_cell_count'})

In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22083 entries, 0 to 22082
Data columns (total 37 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Patient Age                                       20656 non-null  float64
 1   Genes in mother's side                            22083 non-null  object 
 2   Inherited from father                             21777 non-null  object 
 3   Maternal gene                                     19273 non-null  object 
 4   Paternal gene                                     22083 non-null  object 
 5   Blood cell count (mcL)                            22083 non-null  float64
 6   Mother's age                                      16047 non-null  float64
 7   Father's age                                      16097 non-null  float64
 8   Respiratory Rate (breaths/min)                    19934 non-null  object 
 9   Heart Rate (rates

In [10]:
# Fill na values
df_train['Patient Age'] = df_train['Patient Age'].fillna(value = df_train['Patient Age'].median())

In [11]:
df_train['Inherited from father'] = df_train['Inherited from father'].fillna(method = "backfill")

In [12]:
df_train['Maternal gene'] = df_train['Maternal gene'].fillna(method = 'backfill')

In [13]:
df_train["Mother's age"] = df_train["Mother's age"].fillna(value = df_train["Mother's age"].median())

In [14]:
df_train["Father's age"] = df_train["Father's age"].fillna(value = df_train["Father's age"].median())

In [15]:
df_train["Respiratory Rate (breaths/min)"] = df_train["Respiratory Rate (breaths/min)"].fillna(method = 'backfill')

In [16]:
df_train["Heart Rate (rates/min"] = df_train["Heart Rate (rates/min"].fillna(method = 'backfill')

In [17]:
df_train['Test 1'] = df_train['Test 1'].fillna(method = 'backfill')

In [18]:
df_train['Test 2'] = df_train['Test 2'].fillna(method = 'backfill')

In [19]:
df_train['Test 3'] = df_train['Test 3'].fillna(method = 'backfill')

In [20]:
df_train['Test 4'] = df_train['Test 4'].fillna(method = 'backfill')

In [21]:
df_train['Test 5'] = df_train['Test 5'].fillna(method = 'backfill')

In [22]:
df_train['Parental consent'] = df_train['Parental consent'].fillna(value = 'No')

In [23]:
df_train['Follow-up'] = df_train['Follow-up'].fillna(method = 'backfill')

In [24]:
df_train['Gender'] = df_train['Gender'].fillna(method = 'backfill')

In [25]:
df_train['Birth asphyxia'] = df_train['Gender'].fillna(method = 'backfill')

In [26]:
df_train['Autopsy shows birth defect (if applicable)'] = df_train['Autopsy shows birth defect (if applicable)'].fillna(value = 'Not applicable')

In [27]:
df_train['Folic acid details (peri-conceptional)'] = df_train['Folic acid details (peri-conceptional)'].fillna(method = 'backfill')

In [28]:
df_train['H/O serious maternal illness'] = df_train['H/O serious maternal illness'].fillna(method = 'backfill')

In [29]:
df_train['H/O radiation exposure (x-ray)'] = df_train['H/O radiation exposure (x-ray)'].fillna(value = 'Not applicable')

In [30]:
df_train['H/O substance abuse'] = df_train['H/O substance abuse'].fillna(value = 'Not applicable')

In [31]:
df_train['Assisted conception IVF/ART'] = df_train['Assisted conception IVF/ART'].fillna(method = 'backfill')

In [32]:
df_train['History of anomalies in previous pregnancies'] = df_train['History of anomalies in previous pregnancies'].fillna(method = 'backfill')

In [33]:
df_train['No. of previous abortion'] = df_train['No. of previous abortion'].fillna(value = 0.)

In [34]:
df_train['Birth defects'] = df_train['Birth defects'].fillna(value = 'None')

In [35]:
df_train['White Blood cell count (thousand per microliter)'] = df_train['White Blood cell count (thousand per microliter)'].fillna(value = df_train['White Blood cell count (thousand per microliter)'].median())

In [36]:
df_train['Blood test result'] = df_train['Blood test result'].fillna(method = 'backfill')

In [37]:
df_train['Symptom 1'] = df_train['Symptom 1'].fillna(method = 'backfill')

In [38]:
df_train['Symptom 2'] = df_train['Symptom 2'].fillna(method = 'backfill')

In [39]:
df_train['Symptom 3'] = df_train['Symptom 3'].fillna(method = 'backfill')

In [40]:
df_train['Symptom 4'] = df_train['Symptom 4'].fillna(method = 'backfill')

In [41]:
df_train['Symptom 5'] = df_train['Symptom 5'].fillna(method = 'backfill')

In [42]:
df_train = df_train.dropna()

In [43]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [44]:
# verify all na values have been filled
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18047 entries, 0 to 22082
Data columns (total 37 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Patient Age                                       18047 non-null  float64
 1   Genes in mother's side                            18047 non-null  object 
 2   Inherited from father                             18047 non-null  object 
 3   Maternal gene                                     18047 non-null  object 
 4   Paternal gene                                     18047 non-null  object 
 5   Blood cell count (mcL)                            18047 non-null  float64
 6   Mother's age                                      18047 non-null  float64
 7   Father's age                                      18047 non-null  float64
 8   Respiratory Rate (breaths/min)                    18047 non-null  object 
 9   Heart Rate (rates

In [45]:
df_train['Genetic Disorder'].unique()

array(['Mitochondrial genetic inheritance disorders',
       'Multifactorial genetic inheritance disorders',
       'Single-gene inheritance diseases'], dtype=object)

In [46]:
df_train['Disorder Subclass'].unique()

array(["Leber's hereditary optic neuropathy", 'Diabetes',
       'Leigh syndrome', 'Cancer', 'Cystic fibrosis', 'Tay-Sachs',
       'Hemochromatosis', 'Mitochondrial myopathy', "Alzheimer's"],
      dtype=object)

In [47]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9465 entries, 0 to 9464
Data columns (total 35 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Patient Age                                       9465 non-null   int64  
 1   Genes in mother's side                            9465 non-null   object 
 2   Inherited from father                             8914 non-null   object 
 3   Maternal gene                                     5742 non-null   object 
 4   Paternal gene                                     9465 non-null   object 
 5   Blood cell count (mcL)                            9465 non-null   float64
 6   Mother's age                                      9465 non-null   int64  
 7   Father's age                                      9465 non-null   int64  
 8   Respiratory Rate (breaths/min)                    6579 non-null   object 
 9   Heart Rate (rates/m

In [48]:
from tensorflow import constant

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
# standardize variables and do one hot encoding 
df_train['Patient Age'] = (df_train['Patient Age'] - df_train['Patient Age'].mean())/df_train['Patient Age'].std()

In [90]:
df_train["Mother's age"] = (df_train["Mother's age"] - df_train["Mother's age"].mean())/df_train["Mother's age"].std()
df_train["Father's age"] = (df_train["Father's age"] - df_train["Father's age"].mean())/df_train["Father's age"].std()
df_train["Blood cell count (mcL)"] = (df_train["Blood cell count (mcL)"] - df_train["Blood cell count (mcL)"].mean())/df_train["Blood cell count (mcL)"].std()
df_train["Test 1"] = (df_train["Test 1"] - df_train["Test 1"].mean())/df_train["Test 1"].std()
df_train["Test 2"] = (df_train["Test 2"] - df_train["Test 2"].mean())/df_train["Test 2"].std()
df_train["Test 3"] = (df_train["Test 3"] - df_train["Test 3"].mean())/df_train["Test 3"].std()
df_train["Test 4"] = (df_train["Test 4"] - df_train["Test 4"].mean())/df_train["Test 4"].std()
df_train["Test 5"] = (df_train["Test 5"] - df_train["Test 5"].mean())/df_train["Test 5"].std()
df_train["No. of previous abortion"] = (df_train["No. of previous abortion"] - df_train["No. of previous abortion"].mean())/df_train["No. of previous abortion"].std()
df_train["White Blood cell count (thousand per microliter)"] = (df_train["White Blood cell count (thousand per microliter)"] - df_train["White Blood cell count (thousand per microliter)"].mean())/df_train["White Blood cell count (thousand per microliter)"].std()











In [71]:
X = df_train.iloc[:,:35].values
X2 = df_train.iloc[:,:36].values
y = df_train.iloc[:,35:36]
y2 = df_train.iloc[:,36]

In [91]:
X = df_train.drop(['Genetic Disorder','Disorder Subclass'], axis = 1)
X2 = df_train.drop(['Disorder Subclass'], axis = 1)
y1 = df_train['Genetic Disorder']
y2 = df_train['Disorder Subclass']

In [92]:
df_dummies1 = pd.get_dummies(X, drop_first = True)
df_dummies2 = pd.get_dummies(X2, drop_first = True)
print(df_dummies1.info())
print(df_dummies2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18047 entries, 0 to 22082
Data columns (total 46 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Patient Age                                                18047 non-null  float64
 1   Blood cell count (mcL)                                     18047 non-null  float64
 2   Mother's age                                               18047 non-null  float64
 3   Father's age                                               18047 non-null  float64
 4   Test 1                                                     0 non-null      float64
 5   Test 2                                                     0 non-null      float64
 6   Test 3                                                     0 non-null      float64
 7   Test 4                                                     0 non-null      float64
 8   Test 5

In [74]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()
#y2 = ohe.fit_transform(y2).toarray()

### Genetic Disorder Neural Network

In [93]:
dummie_y1 = pd.get_dummies(y1, drop_first = True)
dummie_y2 = pd.get_dummies(y2, drop_first = True)
dummie_y1.head()

,Multifactorial genetic inheritance disorders,Single-gene inheritance diseases
0,0,0
2,1,0
3,0,0
4,1,0
5,0,1


In [133]:
# train test split data
X_train, X_test, y_train, y_test = train_test_split(df_dummies1, dummie_y1, test_size = 0.10, random_state = 42)

In [134]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16242, 46), (1805, 46), (16242, 2), (1805, 2))

In [78]:
from keras.layers import Dropout

In [79]:
def baseline_model():
    model = Sequential()
    model.add(Dense(1024, input_dim = 139, activation = 'relu' ))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
    return model

In [80]:
import keras
from keras.layers import BatchNormalization

In [135]:
def baseline_model2():
    model = Sequential()
    optimizer = ts.keras.optimizers.Adam(learning_rate=0.00001)
    model.add(Dense(384, input_dim = 46, activation = 'relu' ))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy',optimizer = optimizer,metrics = ['accuracy'])
    return model
#normalize the numeric data and add learning rate

In [136]:
estimator1 = KerasClassifier(build_fn = baseline_model2, epochs = 20, batch_size = 512, verbose = 1)

/var/folders/17/zt77bfnj4gl2n1fywj7v7y180000gn/T/ipykernel_69354/193195952.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  estimator1 = KerasClassifier(build_fn = baseline_model2, epochs = 20, batch_size = 512, verbose = 1)


In [83]:
model = Sequential()
optimizer = ts.keras.optimizers.Adam(learning_rate=0.00001)
model.add(Dense(384, input_dim = 139, activation = 'relu' ))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(32, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy',optimizer = optimizer,metrics = ['accuracy'])

2022-01-07 16:36:46.298601: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [89]:
X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')

ValueError: could not convert string to float: 'Yes'

In [88]:
model.fit(X_train, y_train, epochs = 200, batch_size = 512)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [84]:
y_pred = model.predict(X_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [87]:
from sklearn.metrics import accuracy_score
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))
accuracy_score(y_pred, y_test)

KeyError: 0

In [98]:
kfold = KFold(n_splits = 10, shuffle = True)

In [137]:
results = cross_val_score(estimator1, X_train, y_train, cv=kfold)

Epoch 1/20
29/29 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.6152
Epoch 2/20
29/29 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.6152
Epoch 3/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6152
Epoch 4/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6152
Epoch 5/20
29/29 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.6152
Epoch 6/20
29/29 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.6152
Epoch 7/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6152
Epoch 8/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6152
Epoch 9/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6152
Epoch 10/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6152
Epoch 11/20
29/29 [=============================

29/29 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.6144
Epoch 3/20
29/29 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.6144
Epoch 4/20
29/29 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.6144
Epoch 5/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6144
Epoch 6/20
29/29 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.6144
Epoch 7/20
29/29 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.6144
Epoch 8/20
29/29 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.6144
Epoch 9/20
29/29 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.6144
Epoch 10/20
29/29 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.6144
Epoch 11/20
29/29 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.6144
Epoch 12/20
29/29 [==============================]

29/29 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.6160
Epoch 4/20
29/29 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.6160
Epoch 5/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6160
Epoch 6/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6160
Epoch 7/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6160
Epoch 8/20
29/29 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.6160
Epoch 9/20
29/29 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6160
Epoch 10/20
29/29 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.6160
Epoch 11/20
29/29 [==============================] - 0s 15ms/step - loss: nan - accuracy: 0.6160
Epoch 12/20
29/29 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.6160
Epoch 13/20
29/29 [==============================] -

In [103]:
print('Baseline: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))

Baseline: 61.44% (1.92%)


### Subdisorder Neural Network

In [119]:
# train test split data
X2_train, X2_test, y2_train, y2_test = train_test_split(df_dummies2, dummie_y2, test_size = 0.25, random_state = 42)

In [120]:
X2_train.shape, X2_test.shape, y2_train.shape, y2_test.shape

((13535, 48), (4512, 48), (13535, 8), (4512, 8))

In [106]:
X2_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13535 entries, 20742 to 19340
Data columns (total 48 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Patient Age                                                    13535 non-null  float64
 1   Blood cell count (mcL)                                         13535 non-null  float64
 2   Mother's age                                                   13535 non-null  float64
 3   Father's age                                                   13535 non-null  float64
 4   Test 1                                                         0 non-null      float64
 5   Test 2                                                         0 non-null      float64
 6   Test 3                                                         0 non-null      float64
 7   Test 4                                                

In [129]:
def baseline_model3():
    model = Sequential()
    optimizer = ts.keras.optimizers.Adam(learning_rate=0.1)
    model.add(Dense(384, input_dim = 48, activation = 'relu' ))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(8, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy',optimizer = optimizer,metrics = ['accuracy'])
    return model

In [130]:
estimator2 = KerasClassifier(build_fn = baseline_model3, epochs = 20, batch_size = 512, verbose = 1)

/var/folders/17/zt77bfnj4gl2n1fywj7v7y180000gn/T/ipykernel_69354/302185886.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  estimator2 = KerasClassifier(build_fn = baseline_model3, epochs = 20, batch_size = 512, verbose = 1)


In [131]:
results2 = cross_val_score(estimator2, X2_train, y2_train, cv=kfold)

Epoch 1/20
24/24 [==============================] - 1s 13ms/step - loss: nan - accuracy: 0.0122
Epoch 2/20
24/24 [==============================] - 0s 15ms/step - loss: nan - accuracy: 0.0122
Epoch 3/20
24/24 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.0122
Epoch 4/20
24/24 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0122
Epoch 5/20
24/24 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0122
Epoch 6/20
24/24 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.0122
Epoch 7/20
24/24 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.0122
Epoch 8/20
24/24 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.0122
Epoch 9/20
24/24 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.0122
Epoch 10/20
24/24 [==============================] - 0s 13ms/step - loss: nan - accuracy: 0.0122
Epoch 11/20
24/24 [======================

24/24 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.0128
Epoch 3/20
24/24 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.0128
Epoch 4/20
24/24 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.0128
Epoch 5/20
24/24 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.0128
Epoch 6/20
24/24 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.0128
Epoch 7/20
24/24 [==============================] - 0s 16ms/step - loss: nan - accuracy: 0.0128
Epoch 8/20
24/24 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.0128
Epoch 9/20
24/24 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.0128
Epoch 10/20
24/24 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.0128
Epoch 11/20
24/24 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.0128
Epoch 12/20
24/24 [==============================] 

24/24 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.0132
Epoch 4/20
24/24 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.0132
Epoch 5/20
24/24 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0132
Epoch 6/20
24/24 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.0132
Epoch 7/20
24/24 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0132
Epoch 8/20
24/24 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0132
Epoch 9/20
24/24 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0132
Epoch 10/20
24/24 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.0132
Epoch 11/20
24/24 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0132
Epoch 12/20
24/24 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.0132
Epoch 13/20
24/24 [==============================] - 0s 8m

In [132]:
print('Baseline: %.2f%% (%.2f%%)' % (results2.mean()*100, results2.std()*100))

Baseline: 1.29% (0.29%)


### Test Set for Genetic Disorder

In [92]:
test_results1 = cross_val_score(estimator1, X_test, y_test, cv=kfold)

Epoch 1/20
8/8 [==============================] - 1s 14ms/step - loss: 0.6329 - accuracy: 0.4887
Epoch 2/20
8/8 [==============================] - 0s 13ms/step - loss: 0.6159 - accuracy: 0.4911
Epoch 3/20
8/8 [==============================] - 0s 13ms/step - loss: 0.5949 - accuracy: 0.5047
Epoch 4/20
8/8 [==============================] - 0s 14ms/step - loss: 0.6117 - accuracy: 0.4938
Epoch 5/20
8/8 [==============================] - 0s 13ms/step - loss: 0.5866 - accuracy: 0.5039
Epoch 6/20
8/8 [==============================] - 0s 12ms/step - loss: 0.6071 - accuracy: 0.5030
Epoch 7/20
8/8 [==============================] - 0s 12ms/step - loss: 0.6385 - accuracy: 0.4961
Epoch 8/20
8/8 [==============================] - 0s 13ms/step - loss: 0.6129 - accuracy: 0.5071
Epoch 9/20
8/8 [==============================] - 0s 12ms/step - loss: 0.6059 - accuracy: 0.5062
Epoch 10/20
8/8 [==============================] - 0s 13ms/step - loss: 0.6196 - accuracy: 0.5071
Epoch 11/20
8/8 [============

8/8 [==============================] - 1s 19ms/step - loss: 0.6971 - accuracy: 0.4979
Epoch 2/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6959 - accuracy: 0.4979
Epoch 3/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6837 - accuracy: 0.5016
Epoch 4/20
8/8 [==============================] - 0s 12ms/step - loss: 0.6755 - accuracy: 0.4878
Epoch 5/20
8/8 [==============================] - 0s 10ms/step - loss: 0.6689 - accuracy: 0.4905
Epoch 6/20
8/8 [==============================] - 0s 12ms/step - loss: 0.6505 - accuracy: 0.5041
Epoch 7/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6813 - accuracy: 0.4878
Epoch 8/20
8/8 [==============================] - 0s 10ms/step - loss: 0.6729 - accuracy: 0.4999
Epoch 9/20
8/8 [==============================] - 0s 12ms/step - loss: 0.6562 - accuracy: 0.4977
Epoch 10/20
8/8 [==============================] - 0s 13ms/step - loss: 0.6752 - accuracy: 0.4957
Epoch 11/20
8/8 [=======================

8/8 [==============================] - 1s 16ms/step - loss: 0.6909 - accuracy: 0.4910
Epoch 2/20
8/8 [==============================] - 0s 10ms/step - loss: 0.7090 - accuracy: 0.4898
Epoch 3/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6930 - accuracy: 0.4913
Epoch 4/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6515 - accuracy: 0.4964
Epoch 5/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6538 - accuracy: 0.5009
Epoch 6/20
8/8 [==============================] - 0s 12ms/step - loss: 0.6752 - accuracy: 0.4888
Epoch 7/20
8/8 [==============================] - 0s 13ms/step - loss: 0.6715 - accuracy: 0.4858
Epoch 8/20
8/8 [==============================] - 0s 12ms/step - loss: 0.6716 - accuracy: 0.4935
Epoch 9/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6468 - accuracy: 0.5048
Epoch 10/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6304 - accuracy: 0.5080
Epoch 11/20
8/8 [=======================